In [53]:
from model import *

In [54]:
gin.parse_config_file("config/gpt2-small.gin")
config = GPTConfig()
print(config)

GPTConfig(block_size=1024, vocab_size=100288, n_layer=16, n_head=16, n_embd=1024, attn_dropout=0.0, pad_token_id=100278, batch_size=4, epochs=2, clip_grad_norm_val=1.0, training_backend='nccl', learning_rate=0.0003, eta_min=3e-06, weight_decay=0.0001, eps=1e-08, betas=(0.9, 0.97), base_theta=10000.0, scale_factor=1.0, gradient_accumulation_steps=8, warmup_steps=None, warmup_steps_ratio=0.15, total_steps=None, steps_per_epoch=None, dtype=torch.bfloat16, fused_optimizer=True, do_init_params=True, rng_seed=42, rng_device=device(type='cpu'), model_device=device(type='cpu'), rng_generator=<torch._C.Generator object at 0x0000019880402470>)


In [55]:
model = GPT(config)
# model.to(torch.bfloat16)

In [56]:
for pn, p in model.named_parameters():
    if p.requires_grad:
        print(pn, p.shape)

transformer.wte.weight torch.Size([100288, 1024])
transformer.h.0.ln_1.weight torch.Size([1024])
transformer.h.0.ln_1.bias torch.Size([1024])
transformer.h.0.attn.qkv_proj.weight torch.Size([3072, 1024])
transformer.h.0.attn.qkv_proj.bias torch.Size([3072])
transformer.h.0.attn.o_proj.weight torch.Size([1024, 1024])
transformer.h.0.attn.o_proj.bias torch.Size([1024])
transformer.h.0.ln_2.weight torch.Size([1024])
transformer.h.0.ln_2.bias torch.Size([1024])
transformer.h.0.mlp.gate_proj.weight torch.Size([4096, 1024])
transformer.h.0.mlp.gate_proj.bias torch.Size([4096])
transformer.h.0.mlp.up_proj.weight torch.Size([4096, 1024])
transformer.h.0.mlp.up_proj.bias torch.Size([4096])
transformer.h.0.mlp.down_proj.weight torch.Size([1024, 4096])
transformer.h.0.mlp.down_proj.bias torch.Size([1024])
transformer.h.1.ln_1.weight torch.Size([1024])
transformer.h.1.ln_1.bias torch.Size([1024])
transformer.h.1.attn.qkv_proj.weight torch.Size([3072, 1024])
transformer.h.1.attn.qkv_proj.bias torch

In [4]:
optimizer = model.configure_optimizers()

num decayed parameter tensors: 81, with 371,130,368 parameters
num non-decayed parameter tensors: 147, with 380,864 parameters
using fused AdamW: True


In [4]:
# count number of parameters in terms of billions
num_params = sum([param.nelement() for param in model.parameters()])
num_params / 1e6

371.511232

In [5]:
# torch.save(model.state_dict(), "model.pth")

In [24]:
import tiktoken

gpt4_tokenizer = tiktoken.get_encoding("cl100k_base")
# gpt4_tokenizer.n_vocab

In [21]:
special_tokens = {
    "<|startoftext|>": gpt4_tokenizer.n_vocab,
    "<|PAD|>": gpt4_tokenizer.n_vocab + 1,
    "<|TASK|>": gpt4_tokenizer.n_vocab + 2,
    "<|MASK|>": gpt4_tokenizer.n_vocab + 3,
    "<|CLS|>": gpt4_tokenizer.n_vocab + 4,
    "<|USER|>": gpt4_tokenizer.n_vocab + 5,
    "<|ASSISTANT|>": gpt4_tokenizer.n_vocab + 6,
    "<|SYSTEM|>": gpt4_tokenizer.n_vocab + 7,
    "<|im_start|>": gpt4_tokenizer.n_vocab + 8,
    "<|im_end|>": gpt4_tokenizer.n_vocab + 9,
    "<|im_sep|>": gpt4_tokenizer.n_vocab + 10,
    }


In [28]:
custom_tokenizer = tiktoken.Encoding(
    name="custom",
    pat_str=gpt4_tokenizer._pat_str,
    mergeable_ranks=gpt4_tokenizer._mergeable_ranks,
    special_tokens={**gpt4_tokenizer._special_tokens, **special_tokens},
)

In [30]:
custom_tokenizer._special_tokens

{'<|endoftext|>': 100257,
 '<|fim_prefix|>': 100258,
 '<|fim_middle|>': 100259,
 '<|fim_suffix|>': 100260,
 '<|endofprompt|>': 100276,
 '<|startoftext|>': 100277,
 '<|PAD|>': 100278,
 '<|TASK|>': 100279,
 '<|MASK|>': 100280,
 '<|CLS|>': 100281,
 '<|USER|>': 100282,
 '<|ASSISTANT|>': 100283,
 '<|SYSTEM|>': 100284,
 '<|im_start|>': 100285,
 '<|im_end|>': 100286,
 '<|im_sep|>': 100287}

In [46]:
import torch
import torch.nn as nn
a = nn.Embedding(2, 4, padding_idx=1)

In [49]:
a(torch.tensor([1]))

tensor([[0., 0., 0., 0.]], grad_fn=<EmbeddingBackward0>)

1.0

In [ ]:
L.Trainer()

"b'1'"

In [ ]:
vocab = custom_tokenizer._mergeable_ranks
special_tokens = custom_tokenizer._special_tokens

import json
full_vocab = {**vocab, **special_tokens}
with open("vocab.json", "w") as f:
    json.dump(full_vocab, f)

TypeError: a bytes-like object is required, not 'str'

In [6]:
# sample input
text = torch.randint(0, 100288, (1, 1024))
text.shape

torch.Size([1, 1024])

In [10]:
output = model(text)
# output[0].shape

In [14]:
model.transformer.wte.weight.shape

torch.Size([100288, 1024])

In [ ]:
# import torch
# import torch.nn.functional as F
# from transformers import AutoTokenizer



# def generate(model,
#              tokenizer,
#              prompt: str,
#              n_tokens_to_gen: int = 200,
#              sample: bool = True,
#              top_k: int = 40):
#     model.eval()

#     input_ids = tokenizer(prompt, return_tensors='pt').input_ids

#     for token_n in range(n_tokens_to_gen):
#         with torch.no_grad():
#             indices_to_input = input_ids
#             next_token_logits, loss = model(indices_to_input)
#             next_token_logits = next_token_logits[:, -1]

#         probs = F.softmax(next_token_logits, dim=-1)
#         (batch, vocab_size) = probs.shape

#         if top_k is not None:
#             (values, indices) = torch.topk(probs, k=top_k)
#             probs[probs < values[:, -1, None]] = 0
#             probs = probs / probs.sum(axis=1, keepdims=True)

#         if sample:
#             next_indices = torch.multinomial(probs, num_samples=1)
#         else:
#             next_indices = torch.argmax(probs, dim=-1)[:, None]

#         input_ids = torch.cat([input_ids, next_indices], dim=1)

#     output_completions = [tokenizer.decode(output.tolist()) for output in input_ids][0]

#     return output_completions